In [14]:
#%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
%pip install transformers requests beautifulsoup4 pandas numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [3]:
torch.__version__

'2.6.0+cpu'

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

c:\Users\Marcell Suyanto\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [36]:
#site = str(input("Site URL: "))
site = 'https://www.yelp.com/biz/%E6%BE%B3%E6%B4%B2%E7%89%9B%E5%A5%B6%E5%85%AC%E5%8F%B8-%E9%A6%99%E6%B8%AF-2?osq=Restaurants'
page = requests.get(site)
soup = BeautifulSoup(page.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [38]:
df = pd.DataFrame(np.array(reviews), columns=['reviews'])

In [39]:
df.head()

,reviews
0,Hong Kong breakfasts are a vibe! If you're loo...
1,it's worth a visit while in kowloonfor the lov...
2,"The hype is real, I've heard many good things ..."
3,Excellent HK-style cafe. Both egg custards wer...
4,Arrived on a Monday at 745am and was second in...


In [22]:
def sentiment(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)  
    return int(torch.argmax(result.logits)) + 1

In [23]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment(x))
df['reviews']

0    Hong Kong breakfasts are a vibe! If you're loo...
1    it's worth a visit while in kowloonfor the lov...
2    The hype is real, I've heard many good things ...
3    Excellent HK-style cafe. Both egg custards wer...
4    Arrived on a Monday at 745am and was second in...
5    Australia Dairy Company (澳牛) is the place I mi...
6    I have never eaten so fast at a restaurant. Pl...
7    Quick service fast eat quality food. You are i...
8    Who care if the lines are long when the turnov...
9    You really need to know what you want to order...
Name: reviews, dtype: object

In [24]:
df

,reviews,sentiment
0,Hong Kong breakfasts are a vibe! If you're loo...,5
1,it's worth a visit while in kowloonfor the lov...,4
2,"The hype is real, I've heard many good things ...",4
3,Excellent HK-style cafe. Both egg custards wer...,5
4,Arrived on a Monday at 745am and was second in...,5
5,Australia Dairy Company (澳牛) is the place I mi...,5
6,I have never eaten so fast at a restaurant. Pl...,5
7,Quick service fast eat quality food. You are i...,5
8,Who care if the lines are long when the turnov...,3
9,You really need to know what you want to order...,4


In [25]:
mean_sentiment = df['sentiment'].mean()
highest_sentiment = df['sentiment'].max()
lowest_sentiment = df['sentiment'].min()
median_sentiment = df['sentiment'].median()

print(f"Mean: {mean_sentiment}")
print(f"Highest Sentiment: {highest_sentiment}")
print(f"Lowest Sentiment: {lowest_sentiment}")
print(f"Median: {median_sentiment}")
print(f"Number of reviews: {len(df)}")

Mean: 4.5
Highest Sentiment: 5
Lowest Sentiment: 3
Median: 5.0
Number of reviews: 10
